# Session 11: Time-Varying Parameter VAR (Part 2)
# TVP-VAR Models and Applications

## Summer School: Time Series Methods for Finance and Economics

### Learning Objectives

By the end of this session, you will be able to:
1. Specify full TVP-VAR models with stochastic volatility
2. Understand the computational challenges of TVP-VAR estimation
3. Implement simplified TVP-VAR models
4. Calculate time-varying impulse response functions
5. Analyze dynamic spillovers and connectedness
6. Apply TVP-VAR to financial contagion
7. Interpret structural change in multivariate relationships
8. Integrate time series methods for complete analysis

### Prerequisites
- Session 8: Vector Autoregression (VAR)
- Session 10: TVP Models and Kalman Filter
- Understanding of MCMC methods (helpful but not required)

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.linalg import block_diag, cholesky, inv
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.precision', 4)

## 1. TVP-VAR Model Specification

### 1.1 Full TVP-VAR Model

**Primiceri (2005) specification**:

$$\mathbf{y}_t = \mathbf{X}_t \boldsymbol{\beta}_t + \mathbf{A}_t^{-1} \boldsymbol{\Sigma}_t \boldsymbol{\epsilon}_t, \quad \boldsymbol{\epsilon}_t \sim N(\mathbf{0}, \mathbf{I})$$

where:
- $\mathbf{y}_t$: $N \times 1$ vector of variables
- $\mathbf{X}_t$: $1 \times K$ matrix of lags (RHS variables)
- $\boldsymbol{\beta}_t$: $K \times N$ matrix of time-varying coefficients
- $\mathbf{A}_t$: Lower triangular matrix (contemporaneous relations)
- $\boldsymbol{\Sigma}_t$: Diagonal matrix of stochastic volatilities

### 1.2 Time-Varying Parameters

**VAR coefficients** (random walk):
$$\boldsymbol{\beta}_t = \boldsymbol{\beta}_{t-1} + \boldsymbol{\nu}_t, \quad \boldsymbol{\nu}_t \sim N(\mathbf{0}, \mathbf{Q})$$

**Contemporaneous relations**:
$$\mathbf{a}_t = \mathbf{a}_{t-1} + \boldsymbol{\zeta}_t, \quad \boldsymbol{\zeta}_t \sim N(\mathbf{0}, \mathbf{S})$$

where $\mathbf{a}_t$ stacks lower triangular elements of $\mathbf{A}_t^{-1}$.

**Stochastic volatility** (log variance):
$$\log \sigma_{i,t}^2 = \log \sigma_{i,t-1}^2 + \eta_{i,t}, \quad \eta_{i,t} \sim N(0, w_i)$$

### 1.3 Interpretation

**Three sources of variation**:
1. **$\boldsymbol{\beta}_t$**: Changing VAR coefficients (lag responses)
2. **$\mathbf{A}_t$**: Changing contemporaneous relations
3. **$\boldsymbol{\Sigma}_t$**: Changing volatilities (uncertainty)

**Economic meaning**:
- Monetary policy transmission can change
- Shock propagation varies over time
- Forecast uncertainty evolves
- Crisis periods have different dynamics

### 1.4 Computational Challenges

**High dimensionality**:
- For $N=3$, $p=2$: 18 VAR coefficients per equation
- Plus contemporaneous and volatility parameters
- Total: 100+ time-varying parameters

**Estimation**:
- Maximum likelihood infeasible
- Bayesian MCMC methods (Gibbs sampling)
- Requires priors on $\mathbf{Q}$, $\mathbf{S}$, $\mathbf{W}$
- Computationally intensive

**Simplifications for this session**:
- Focus on time-varying $\boldsymbol{\beta}_t$
- Constant volatility or simple structure
- Rolling VAR as approximation

In [ ]:
# Download multivariate data
print("\n" + "="*60)
print("Downloading Multivariate Financial Data")
print("="*60)

# Get multiple asset classes for spillover analysis
tickers = ['SPY', 'TLT', 'GLD']  # Stocks, Bonds, Gold
data = yf.download(tickers, start='2008-01-01', end='2024-01-01', progress=False)['Close']
data.columns = ['Stocks', 'Bonds', 'Gold']
data = data.dropna()

# Calculate returns
returns = data.pct_change().dropna() * 100

print(f"\nData period: {returns.index[0].date()} to {returns.index[-1].date()}")
print(f"Observations: {len(returns)}")
print(f"\nAssets:")
print("  • SPY: S&P 500 ETF (stocks)")
print("  • TLT: 20+ Year Treasury Bond ETF")
print("  • GLD: Gold ETF")
print("\nThis period includes:")
print("  • Financial crisis (2008-2009)")
print("  • European debt crisis (2011-2012)")
print("  • COVID-19 pandemic (2020)")
print("  • Post-pandemic recovery (2021-2023)")

print("\nDescriptive Statistics:")
print(returns.describe().round(4))

print("\nCorrelation Matrix:")
print(returns.corr().round(4))

In [ ]:
# Visualize the data
fig, axes = plt.subplots(4, 1, figsize=(15, 14))

# Returns
returns.plot(ax=axes[0], linewidth=1, alpha=0.7)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0].set_title('Daily Returns: Stocks, Bonds, Gold', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Return (%)', fontsize=11)
axes[0].legend(loc='best', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Add crisis periods
crisis_periods = [
    ('2008-09-01', '2009-03-31', 'Financial Crisis', 'red'),
    ('2020-02-01', '2020-04-30', 'COVID-19', 'orange')
]
for start, end, label, color in crisis_periods:
    axes[0].axvspan(pd.Timestamp(start), pd.Timestamp(end), 
                    alpha=0.2, color=color)

# Rolling volatility (60-day)
rolling_vol = returns.rolling(window=60).std() * np.sqrt(252)
rolling_vol.plot(ax=axes[1], linewidth=1.5, alpha=0.7)
axes[1].set_title('Rolling Volatility (60-day, Annualized)', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Volatility (%)', fontsize=11)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

# Rolling correlations
rolling_corr_sb = returns['Stocks'].rolling(window=120).corr(returns['Bonds'])
rolling_corr_sg = returns['Stocks'].rolling(window=120).corr(returns['Gold'])
rolling_corr_bg = returns['Bonds'].rolling(window=120).corr(returns['Gold'])

axes[2].plot(returns.index, rolling_corr_sb, linewidth=2, label='Stocks-Bonds', alpha=0.7)
axes[2].plot(returns.index, rolling_corr_sg, linewidth=2, label='Stocks-Gold', alpha=0.7)
axes[2].plot(returns.index, rolling_corr_bg, linewidth=2, label='Bonds-Gold', alpha=0.7)
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2].set_title('Rolling Correlations (120-day)', fontsize=13, fontweight='bold')
axes[2].set_ylabel('Correlation', fontsize=11)
axes[2].set_ylim(-1, 1)
axes[2].legend(loc='best', fontsize=10)
axes[2].grid(True, alpha=0.3)

# Cumulative returns
cum_returns = (1 + returns/100).cumprod()
cum_returns.plot(ax=axes[3], linewidth=2, alpha=0.7)
axes[3].set_title('Cumulative Returns', fontsize=13, fontweight='bold')
axes[3].set_xlabel('Date', fontsize=11)
axes[3].set_ylabel('Cumulative Return', fontsize=11)
axes[3].legend(loc='best', fontsize=10)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("• Volatility spikes during crises")
print("• Stock-bond correlation changes sign over time")
print("• Gold serves as safe haven (negative correlation with stocks)")
print("• Relationships are clearly time-varying")

## 2. Simplified TVP-VAR: Rolling Window Approach

### 2.1 Rolling Window VAR

**Simple approximation** to full TVP-VAR:
- Estimate VAR over moving window
- Window size $w$ (e.g., 250 days ≈ 1 year)
- At time $t$: use data $[t-w+1, t]$

**Pros**:
- Easy to implement
- No hyperparameters to tune
- Standard VAR software

**Cons**:
- Discrete jumps in estimates
- Arbitrary window choice
- Less efficient than Kalman

### 2.2 Time-Varying Impulse Responses

**At each time $t$**:
1. Estimate VAR on window ending at $t$
2. Calculate IRFs from estimated coefficients
3. Store IRFs for horizon $h$

**Result**: $\text{IRF}_{ij}(h, t)$ - response of $i$ to shock in $j$ at horizon $h$ and time $t$.

### 2.3 Interpretation

**How shock transmission changes**:
- Crisis vs normal times
- Policy regime changes
- Market integration effects
- Structural breaks

In [ ]:
# Implement rolling window VAR
print("\n" + "="*60)
print("Rolling Window VAR Estimation")
print("="*60)

window = 252  # 1 year
max_lags = 5

# Storage for results
rolling_dates = []
rolling_coefs = []  # Store coefficient matrices
rolling_residuals = []

print(f"\nWindow size: {window} days")
print(f"Maximum lags tested: {max_lags}")
print(f"\nEstimating...")

for i in range(window, len(returns)):
    if i % 500 == 0:
        print(f"  Progress: {i}/{len(returns)}")
    
    # Extract window
    window_data = returns.iloc[i-window:i]
    
    try:
        # Fit VAR
        model = VAR(window_data)
        # Select lag order
        lag_order = model.select_order(maxlags=max_lags)
        selected_lag = lag_order.bic
        
        # Estimate
        fitted = model.fit(selected_lag)
        
        # Store results
        rolling_dates.append(returns.index[i])
        rolling_coefs.append(fitted.params)  # Coefficient matrix
        rolling_residuals.append(fitted.resid)
        
    except:
        # Skip if estimation fails
        continue

print(f"\n✓ Completed: {len(rolling_dates)} windows estimated")
print(f"  Start date: {rolling_dates[0].date()}")
print(f"  End date: {rolling_dates[-1].date()}")

In [ ]:
# Extract specific coefficients over time
print("\n" + "="*60)
print("Time-Varying VAR Coefficients")
print("="*60)

# Focus on first lag coefficients
# Structure: [const, L1.Stocks, L1.Bonds, L1.Gold] for each equation

# Extract coefficients for Stocks equation
stocks_on_stocks_l1 = [coef.iloc[1, 0] for coef in rolling_coefs]  # Stocks on Stocks(-1)
stocks_on_bonds_l1 = [coef.iloc[2, 0] for coef in rolling_coefs]   # Stocks on Bonds(-1)
stocks_on_gold_l1 = [coef.iloc[3, 0] for coef in rolling_coefs]    # Stocks on Gold(-1)

# Convert to DataFrame
coef_df = pd.DataFrame({
    'Stocks_on_Stocks': stocks_on_stocks_l1,
    'Stocks_on_Bonds': stocks_on_bonds_l1,
    'Stocks_on_Gold': stocks_on_gold_l1
}, index=rolling_dates)

print("\nStocks Equation - First Lag Coefficients:")
print(coef_df.describe().round(4))

In [ ]:
# Visualize time-varying coefficients
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Stocks on Stocks(-1)
coef_df['Stocks_on_Stocks'].plot(ax=axes[0], linewidth=2, color='blue', alpha=0.7)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[0].axhline(y=coef_df['Stocks_on_Stocks'].mean(), color='red', 
               linestyle='--', linewidth=2, alpha=0.7,
               label=f"Mean: {coef_df['Stocks_on_Stocks'].mean():.3f}")
axes[0].set_title('TVP: Stocks on Stocks(-1)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Coefficient', fontsize=11)
axes[0].legend(loc='best', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Add crisis periods
for start, end, label, color in crisis_periods:
    axes[0].axvspan(pd.Timestamp(start), pd.Timestamp(end), 
                    alpha=0.15, color=color)

# Stocks on Bonds(-1)
coef_df['Stocks_on_Bonds'].plot(ax=axes[1], linewidth=2, color='green', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].axhline(y=coef_df['Stocks_on_Bonds'].mean(), color='red',
               linestyle='--', linewidth=2, alpha=0.7,
               label=f"Mean: {coef_df['Stocks_on_Bonds'].mean():.3f}")
axes[1].set_title('TVP: Stocks on Bonds(-1)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Coefficient', fontsize=11)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

for start, end, label, color in crisis_periods:
    axes[1].axvspan(pd.Timestamp(start), pd.Timestamp(end),
                    alpha=0.15, color=color)

# Stocks on Gold(-1)
coef_df['Stocks_on_Gold'].plot(ax=axes[2], linewidth=2, color='orange', alpha=0.7)
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2].axhline(y=coef_df['Stocks_on_Gold'].mean(), color='red',
               linestyle='--', linewidth=2, alpha=0.7,
               label=f"Mean: {coef_df['Stocks_on_Gold'].mean():.3f}")
axes[2].set_title('TVP: Stocks on Gold(-1)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Date', fontsize=11)
axes[2].set_ylabel('Coefficient', fontsize=11)
axes[2].legend(loc='best', fontsize=10)
axes[2].grid(True, alpha=0.3)

for start, end, label, color in crisis_periods:
    axes[2].axvspan(pd.Timestamp(start), pd.Timestamp(end),
                    alpha=0.15, color=color)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("• Autoregressive coefficient varies substantially")
print("• Bond-stock relationship changes sign over time")
print("• Gold coefficient more stable but with variation")
print("• Crisis periods show different dynamics")

## 3. Time-Varying Impulse Response Functions

### 3.1 Computing Time-Varying IRFs

**For each time $t$**:
1. Use VAR coefficients $\Phi_t^{(1)}, \ldots, \Phi_t^{(p)}$
2. Compute MA representation: $\Psi_0, \Psi_1, \ldots, \Psi_h$
3. Apply Cholesky orthogonalization
4. Extract IRF for each variable pair

**Result**: $\text{IRF}_{ij}(h, t)$

### 3.2 Visualization Strategies

**Heatmaps**: Show IRF evolution over time and horizon

**3D surfaces**: Time × Horizon × Response

**Time slices**: IRFs at specific dates (crisis vs normal)

**Horizon slices**: How immediate vs delayed responses change

### 3.3 Economic Interpretation

**Flight to quality**:
- During crisis: Negative stock shock → positive bond response
- Normal times: Weaker or opposite effect

**Safe haven behavior**:
- Gold response to stock shocks
- Time-varying hedge properties

**Contagion**:
- Spillovers stronger during stress
- Market integration effects

In [ ]:
# Calculate time-varying IRFs
print("\n" + "="*60)
print("Time-Varying Impulse Response Functions")
print("="*60)

# Parameters
irf_horizon = 10
sample_every = 20  # Sample every N periods to reduce computation

# Storage for IRFs
# Structure: time × horizon × response_var × shock_var
n_times = len(range(0, len(rolling_dates), sample_every))
n_vars = len(returns.columns)
tv_irfs = np.zeros((n_times, irf_horizon+1, n_vars, n_vars))
sampled_dates = []

print(f"\nComputing IRFs:")
print(f"  Horizon: {irf_horizon} periods")
print(f"  Sampling: every {sample_every} periods")
print(f"  Total IRFs: {n_times}")

for idx, i in enumerate(range(0, len(rolling_dates), sample_every)):
    if idx % 20 == 0:
        print(f"  Progress: {idx}/{n_times}")
    
    # Get window data
    window_end = rolling_dates[i]
    window_start_idx = returns.index.get_loc(window_end) - window + 1
    window_data = returns.iloc[window_start_idx:window_start_idx+window]
    
    try:
        # Fit VAR
        model = VAR(window_data)
        lag_order = model.select_order(maxlags=max_lags)
        fitted = model.fit(lag_order.bic)
        
        # Calculate IRF
        irf = fitted.irf(irf_horizon)
        
        # Store orthogonalized IRFs
        tv_irfs[idx, :, :, :] = irf.orth_irfs
        sampled_dates.append(window_end)
        
    except:
        # If estimation fails, use NaN
        tv_irfs[idx, :, :, :] = np.nan
        sampled_dates.append(window_end)

print(f"\n✓ IRF calculation complete")
print(f"  Computed {n_times} time-varying IRFs")

In [ ]:
# Visualize time-varying IRFs: Stock shock → Bond response
print("\n" + "="*60)
print("Time-Varying IRF: Stock Shock → Bond Response")
print("="*60)

# Extract: Bonds (index 1) response to Stocks (index 0) shock
irf_stocks_to_bonds = tv_irfs[:, :, 1, 0]  # [time, horizon]

# Create heatmap
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Heatmap
im = axes[0].imshow(irf_stocks_to_bonds.T, aspect='auto', cmap='RdBu_r',
                    origin='lower', extent=[0, len(sampled_dates), 0, irf_horizon],
                    vmin=-0.1, vmax=0.1)
axes[0].set_title('Time-Varying IRF: Bond Response to Stock Shock',
                  fontsize=13, fontweight='bold')
axes[0].set_xlabel('Time', fontsize=11)
axes[0].set_ylabel('Horizon (days)', fontsize=11)

# Add colorbar
cbar = plt.colorbar(im, ax=axes[0])
cbar.set_label('Response', fontsize=11)

# Set x-axis to dates (sample for readability)
n_ticks = 6
tick_indices = np.linspace(0, len(sampled_dates)-1, n_ticks, dtype=int)
axes[0].set_xticks(tick_indices)
axes[0].set_xticklabels([sampled_dates[i].strftime('%Y-%m') for i in tick_indices],
                        rotation=45)

# Plot impact response over time (horizon 0)
axes[1].plot(sampled_dates, irf_stocks_to_bonds[:, 0], linewidth=2,
            color='blue', alpha=0.7, label='Impact (h=0)')
axes[1].plot(sampled_dates, irf_stocks_to_bonds[:, 1], linewidth=2,
            color='green', alpha=0.7, label='1-day (h=1)')
axes[1].plot(sampled_dates, irf_stocks_to_bonds[:, 5], linewidth=2,
            color='orange', alpha=0.7, label='5-day (h=5)')
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Bond Response to Stock Shock at Different Horizons',
                  fontsize=13, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Response', fontsize=11)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("• During crises: Stock shock → positive bond response (flight to safety)")
print("• Normal times: Weaker or opposite relationship")
print("• Response magnitude and persistence vary over time")
print("• Heatmap shows evolution of entire IRF profile")

In [ ]:
# Compare IRFs: Crisis vs Normal Times
print("\n" + "="*60)
print("IRF Comparison: Crisis vs Normal Times")
print("="*60)

# Select specific dates
# Crisis: Financial crisis (2008-10)
# Normal: Pre-crisis (2007-06)

# Find closest dates in sampled_dates
crisis_date = pd.Timestamp('2008-10-15')
normal_date = pd.Timestamp('2007-06-15')

def find_closest_idx(target_date, date_list):
    return min(range(len(date_list)), 
               key=lambda i: abs(date_list[i] - target_date))

crisis_idx = find_closest_idx(crisis_date, sampled_dates)
normal_idx = find_closest_idx(normal_date, sampled_dates)

print(f"\nSelected dates:")
print(f"  Normal: {sampled_dates[normal_idx].date()}")
print(f"  Crisis: {sampled_dates[crisis_idx].date()}")

# Extract IRFs
var_names = returns.columns

fig, axes = plt.subplots(3, 3, figsize=(16, 14))

for i, resp_var in enumerate(var_names):
    for j, shock_var in enumerate(var_names):
        ax = axes[i, j]
        
        # Normal times IRF
        irf_normal = tv_irfs[normal_idx, :, i, j]
        # Crisis times IRF
        irf_crisis = tv_irfs[crisis_idx, :, i, j]
        
        horizons = np.arange(irf_horizon+1)
        ax.plot(horizons, irf_normal, 'b-', linewidth=2, 
               label='Normal', marker='o', markersize=4)
        ax.plot(horizons, irf_crisis, 'r--', linewidth=2,
               label='Crisis', marker='s', markersize=4)
        ax.axhline(y=0, color='black', linestyle=':', alpha=0.5)
        
        if i == 0 and j == 0:
            ax.legend(loc='best', fontsize=9)
        
        ax.set_title(f'{resp_var} ← {shock_var}', fontsize=10, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        if i == 2:
            ax.set_xlabel('Horizon', fontsize=9)
        if j == 0:
            ax.set_ylabel('Response', fontsize=9)

plt.suptitle('Impulse Response Functions: Crisis vs Normal Times',
            fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\nKey Differences:")
print("• Crisis: Larger immediate responses (higher volatility)")
print("• Crisis: Stronger cross-asset spillovers")
print("• Crisis: Stock-Bond negative correlation (flight to quality)")
print("• Normal: More muted responses, faster decay")

## 4. Dynamic Spillovers and Connectedness

### 4.1 Diebold-Yilmaz Spillover Index

**Forecast Error Variance Decomposition** at time $t$ and horizon $h$:

$$\theta_{ij}^{(h, t)} = \frac{\sum_{k=0}^{h-1} (\mathbf{e}_i^\top \Psi_k^{(t)} \mathbf{P}^{(t)} \mathbf{e}_j)^2}{\sum_{k=0}^{h-1} \mathbf{e}_i^\top \Psi_k^{(t)} \Sigma^{(t)} (\Psi_k^{(t)})^\top \mathbf{e}_i}$$

**Spillover index** at time $t$:

$$S^{(t)} = \frac{\sum_{i \neq j} \theta_{ij}^{(h,t)}}{\sum_{i,j} \theta_{ij}^{(h,t)}} \times 100\%$$

**Interpretation**:
- Fraction of forecast error variance from other variables
- $S^{(t)} = 0$: No spillovers (independent)
- $S^{(t)} = 100$: All variation from others

### 4.2 Directional Spillovers

**From others to $i$**:
$$S_{i \leftarrow \bullet}^{(t)} = \frac{\sum_{j \neq i} \theta_{ij}^{(h,t)}}{\sum_j \theta_{ij}^{(h,t)}} \times 100\%$$

**From $i$ to others**:
$$S_{i \rightarrow \bullet}^{(t)} = \frac{\sum_{j \neq i} \theta_{ji}^{(h,t)}}{\sum_j \theta_{ji}^{(h,t)}} \times 100\%$$

**Net spillover**:
$$S_i^{\text{net}(t)} = S_{i \rightarrow \bullet}^{(t)} - S_{i \leftarrow \bullet}^{(t)}$$

**Interpretation**:
- $S_i^{\text{net}} > 0$: Net transmitter of shocks
- $S_i^{\text{net}} < 0$: Net receiver of shocks

### 4.3 Applications

**Financial contagion**:
- Spillovers increase during crises
- Identify systemically important assets

**Market integration**:
- Rising spillovers over time
- Globalization effects

**Risk management**:
- Time-varying diversification benefits
- Dynamic hedging strategies

In [ ]:
# Calculate spillover indices
print("\n" + "="*60)
print("Dynamic Spillover Analysis")
print("="*60)

def calculate_fevd(irf_orth, horizon):
    """
    Calculate FEVD from orthogonalized IRFs.
    irf_orth: [horizon+1, n_vars, n_vars]
    Returns: [n_vars, n_vars] FEVD matrix
    """
    n_vars = irf_orth.shape[1]
    fevd = np.zeros((n_vars, n_vars))
    
    # Sum squared IRFs up to horizon
    for i in range(n_vars):
        mse_i = 0
        for h in range(horizon+1):
            mse_i += np.sum(irf_orth[h, i, :]**2)
        
        for j in range(n_vars):
            contribution = 0
            for h in range(horizon+1):
                contribution += irf_orth[h, i, j]**2
            
            fevd[i, j] = contribution / mse_i if mse_i > 0 else 0
    
    return fevd

def spillover_index(fevd):
    """
    Calculate total spillover index from FEVD matrix.
    """
    n = fevd.shape[0]
    total = np.sum(fevd)
    own = np.trace(fevd)
    cross = total - own
    return (cross / total * 100) if total > 0 else 0

# Calculate spillovers at each time point
horizon_fevd = 5  # Horizon for FEVD
spillovers = []

print(f"\nCalculating spillover indices (horizon={horizon_fevd})...")

for t in range(len(sampled_dates)):
    fevd_t = calculate_fevd(tv_irfs[t], horizon_fevd)
    spillover_t = spillover_index(fevd_t)
    spillovers.append(spillover_t)

spillovers = np.array(spillovers)

print(f"\n✓ Spillover calculation complete")
print(f"\nSpillover Index Statistics:")
print(f"  Mean: {np.nanmean(spillovers):.2f}%")
print(f"  Std:  {np.nanstd(spillovers):.2f}%")
print(f"  Min:  {np.nanmin(spillovers):.2f}%")
print(f"  Max:  {np.nanmax(spillovers):.2f}%")

In [ ]:
# Visualize spillover index over time
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Total spillover index
axes[0].plot(sampled_dates, spillovers, linewidth=2, color='darkred', alpha=0.7)
axes[0].axhline(y=np.nanmean(spillovers), color='blue', linestyle='--',
               linewidth=2, label=f'Mean: {np.nanmean(spillovers):.1f}%')
axes[0].set_title('Total Spillover Index Over Time', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Spillover Index (%)', fontsize=11)
axes[0].legend(loc='best', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Shade crisis periods
for start, end, label, color in crisis_periods:
    axes[0].axvspan(pd.Timestamp(start), pd.Timestamp(end),
                    alpha=0.2, color=color, label=label)

# Add legend for crisis periods (only once)
handles, labels = axes[0].get_legend_handles_labels()
# Remove duplicates
by_label = dict(zip(labels, handles))
axes[0].legend(by_label.values(), by_label.keys(), loc='best', fontsize=10)

# Volatility for comparison
avg_vol = returns.rolling(window=20).std().mean(axis=1) * np.sqrt(252)
# Resample to match sampled_dates
vol_sampled = [avg_vol.loc[:date].iloc[-1] if date in avg_vol.index 
               else np.nan for date in sampled_dates]

axes[1].plot(sampled_dates, vol_sampled, linewidth=2, color='purple', alpha=0.7,
            label='Average Volatility')
axes[1].set_title('Average Volatility (for comparison)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Volatility (%)', fontsize=11)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Findings:")
print("• Spillovers spike during financial crises")
print("• Correlates with market volatility")
print("• Markets become more interconnected under stress")
print("• Diversification benefits reduced during crises")

## 5. Course Integration and Final Synthesis

### 5.1 The Complete Time Series Toolkit

**Sessions 1-4: Univariate Foundations**
- Stationarity and unit roots
- ARIMA models for forecasting
- Exponential smoothing and decomposition
- Model selection and diagnostics

**Sessions 5-7: Volatility Modeling**
- ARIMA extensions (SARIMA, ARIMAX)
- ARCH/GARCH for time-varying volatility
- Multivariate GARCH (DCC)
- Risk measures (VaR, ES)

**Sessions 8-9: Multivariate Models**
- VAR for multiple time series
- Granger causality and spillovers
- Cointegration and VECM
- Long-run equilibrium relationships

**Sessions 10-11: Time-Varying Parameters**
- State space models and Kalman filter
- TVP regression and VAR
- Dynamic impulse responses
- Structural change and regime shifts

### 5.2 Integrated Workflow Example

**Research Question**: How do stock-bond correlations change over time?

**Step 1: Data Preparation** (Sessions 1-2)
- Test for stationarity
- Transform if needed
- Handle missing data
- Outlier detection

**Step 2: Univariate Analysis** (Sessions 3-4)
- Model each series separately
- Understand individual dynamics
- Benchmark forecasts

**Step 3: Volatility Analysis** (Sessions 6-7)
- Estimate GARCH models
- Time-varying volatility
- Risk assessment

**Step 4: Cointegration Test** (Session 9)
- Are series cointegrated?
- If yes: Use VECM
- If no: Use VAR in differences

**Step 5: Constant-Parameter VAR** (Session 8)
- Baseline multivariate model
- Static IRFs and FEVD
- Test for parameter stability

**Step 6: TVP-VAR** (Sessions 10-11)
- Time-varying coefficients
- Dynamic correlations
- Crisis vs normal times
- Spillover evolution

**Step 7: Economic Interpretation**
- Flight to quality during stress
- Changing diversification benefits
- Policy implications
- Trading strategies

### 5.3 Best Practices

**Model Selection**:
- Start simple, add complexity as needed
- Use information criteria
- Out-of-sample validation
- Economic sensibility

**Parameter Stability**:
- Always test for breaks
- Use recursive estimation
- Consider TVP if unstable
- Balance fit vs parsimony

**Forecasting**:
- Use appropriate evaluation metrics
- Multiple horizons
- Density forecasts when possible
- Model averaging

**Interpretation**:
- Connect to economic theory
- Understand limitations
- Uncertainty quantification
- Sensitivity analysis

### 5.4 Advanced Topics and Extensions

**Machine Learning Integration**:
- Neural networks for forecasting
- Regularized TVP-VAR (LASSO, Ridge)
- Tree-based methods
- Ensemble approaches

**High-Frequency Data**:
- Realized measures
- Microstructure effects
- HAR models
- Continuous-time methods

**Mixed-Frequency Models**:
- MIDAS regression
- State space with mixed frequencies
- Nowcasting

**Factor Models**:
- Dynamic factor models
- Factor-augmented VAR
- Big data applications

**Non-Gaussian Methods**:
- Quantile regression
- Copula models
- Extreme value theory

### 5.5 Software and Resources

**Python**:
- statsmodels: Standard time series
- arch: GARCH models
- PyMC: Bayesian estimation
- scikit-learn: ML integration

**R**:
- vars: VAR/VECM
- rugarch: GARCH
- bvartools: Bayesian VAR
- MSBVAR: TVP-VAR

**MATLAB**:
- Econometrics Toolbox
- Bear Toolbox (TVP-VAR)
- RISE Toolbox

**Commercial**:
- EViews
- RATS
- Ox/PcGive

## 6. Final Exercises

### Exercise 1: Complete TVP-VAR Analysis
For a trivariate system of your choice:
1. Estimate constant-parameter VAR
2. Test for parameter stability
3. Estimate rolling window VAR
4. Calculate time-varying IRFs
5. Compare crisis vs normal dynamics
6. Economic interpretation

### Exercise 2: Spillover Analysis
For international stock markets:
1. Download data for US, Europe, Asia
2. Calculate rolling VAR
3. Compute spillover index over time
4. Identify net transmitters vs receivers
5. How does connectedness evolve?
6. Link to major events

### Exercise 3: Crisis Detection
Use TVP-VAR to detect crises:
1. Estimate TVP-VAR on financial data
2. Track changes in parameters
3. Identify rapid changes
4. Compare with known crisis dates
5. Early warning indicators?

### Exercise 4: Forecasting Horse Race
Compare forecasting methods:
1. ARIMA (univariate)
2. VAR (constant parameter)
3. VECM (if cointegrated)
4. Rolling VAR
5. Which performs best? When? Why?

### Exercise 5: Integrated Project
Complete analysis of a research question:
1. Choose economic/financial question
2. Apply appropriate methods from course
3. Test robustness
4. Provide economic interpretation
5. Policy/trading implications
6. Write brief report

In [ ]:
# Space for your solutions to exercises

# Exercise 1:
# Your code here

# Exercise 2:
# Your code here

# Exercise 3:
# Your code here

# Exercise 4:
# Your code here

# Exercise 5:
# Your code here

## References and Further Reading

### Seminal Papers:
1. Primiceri, G.E. (2005). Time varying structural vector autoregressions and monetary policy. *Review of Economic Studies*, 72(3), 821-852.
2. Cogley, T., & Sargent, T.J. (2005). Drifts and volatilities: Monetary policies and outcomes in the post WWII US. *Review of Economic Dynamics*, 8(2), 262-302.
3. Diebold, F.X., & Yilmaz, K. (2009). Measuring financial asset return and volatility spillovers. *Economic Journal*, 119(534), 158-171.
4. Diebold, F.X., & Yilmaz, K. (2014). On the network topology of variance decompositions. *Journal of Econometrics*, 182(1), 119-134.

### Textbooks:
1. Koop, G., & Korobilis, D. (2010). Bayesian multivariate time series methods for empirical macroeconomics. *Foundations and Trends in Econometrics*, 3(4), 267-358.
2. Chan, J.C., Koop, G., Poirier, D.J., & Tobias, J.L. (2019). *Bayesian Econometric Methods* (2nd ed.). Cambridge University Press.
3. Durbin, J., & Koopman, S.J. (2012). *Time Series Analysis by State Space Methods* (2nd ed.). Oxford University Press.

### Applied Papers:
1. Koop, G., Leon-Gonzalez, R., & Strachan, R.W. (2009). On the evolution of the monetary policy transmission mechanism. *Journal of Economic Dynamics and Control*, 33(4), 997-1017.
2. Bańbura, M., Giannone, D., & Reichlin, L. (2010). Large Bayesian vector auto regressions. *Journal of Applied Econometrics*, 25(1), 71-92.
3. Caggiano, G., Castelnuovo, E., & Groshenny, N. (2014). Uncertainty shocks and unemployment dynamics. *Journal of Monetary Economics*, 67, 78-92.

### Software and Implementations:
1. BEAR Toolbox (MATLAB): https://github.com/european-central-bank/BEAR-toolbox
2. bvartools (R): https://cran.r-project.org/package=bvartools
3. PyMC: https://www.pymc.io/
4. Stan: https://mc-stan.org/

### Online Resources:
1. Diebold-Yilmaz Connectedness: https://www.connectednessnetwork.com/
2. Koop's Lecture Notes: http://personal.strath.ac.uk/gary.koop/
3. ECB Working Papers: https://www.ecb.europa.eu/

---

**Instructor Contact**: [Mathis J.F. Mourey. mjfmourey@hhs.nl]

**Office Hours**: [Mon-Fri 9am-5pm]

---

# End of Summer School: Time Series Methods for Finance and Economics

## Thank you for participating!

**Course Summary**:
- 11 comprehensive sessions
- From univariate ARIMA to TVP-VAR
- Theory, implementation, and applications
- Real financial and economic data
- Production-ready code

**What's next?**
- Practice with your own data
- Read papers using these methods
- Contribute to open-source implementations
- Apply to research and industry
- Stay updated with new developments

**Good luck with your time series projects!**